In [ ]:
from typing import List
import uproot
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker
import os

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 20})

In [ ]:
def p_value(data_ts: float, toy_ts: List[float]):
    return np.mean(toy_ts >= data_ts)

def F_statistic(
    ts_low: List[float],
    ts_high: list[float],
    ord_low: int,
    ord_high: int,
    num_bins: int = 10 * 14,
    dim: int = 2,
):
    numerator = (ts_low - ts_high) / (ord_high - ord_low)
    denominator = ts_high / (num_bins - (ord_high + dim))

    return numerator / denominator

In [ ]:
local_cards_dir = "/uscms/home/rkansal/hhcombine/cards/f_tests/23May14/"
test_orders = [0]
test_statistics = {}

for o1 in test_orders:
    tdict = {"toys": {}, "data": {}, "ftoys": {}, "fdata": {}}
    tlabel = f"{o1}"

    for nTF in [o1, o1 + 1]:
        # for nTF1, nTF2 in [(o1, o2)]:
        tflabel = f"{nTF}"

        # test statistics for toys generated by (o1, o2) order model
        file = uproot.concatenate(
            f"{local_cards_dir}/nTF_{nTF}/higgsCombineToys{tlabel}.GoodnessOfFit.mH125.444.root"
        )
        tdict["toys"][tflabel] = np.array(file["limit"])

        # data test statistic
        file = uproot.concatenate(
            f"{local_cards_dir}/nTF_{nTF}/higgsCombineData.GoodnessOfFit.mH125.root"
        )
        tdict["data"][tflabel] = file["limit"][0]

        if not nTF == o1:
            tdict["ftoys"][tflabel] = F_statistic(
                tdict["toys"][tlabel], tdict["toys"][tflabel], o1, nTF
            )
            tdict["fdata"][tflabel] = F_statistic(
                tdict["data"][tlabel], tdict["data"][tflabel], o1, nTF
            )

    test_statistics[tlabel] = tdict